In [3]:
import numpy as np

import os
from pathlib import Path

In [4]:
path = Path(os.getcwd())
root = Path(path.parent.absolute())

model_pre_path = root / 'Models' / 'Pretrained Inception'
model_re_path = root / 'Models' / 'Retrained Inception'
fetching_path = root / 'Shared Preprocessed Objects'

In [5]:
model_re_path / 'HistoryArray.npy'

WindowsPath('C:/Users/nicol/Documents/Mestrado/Matérias/Applications of Deep Learning/Pull Request - Rename/Models/Retrained Inception/HistoryArray.npy')

In [6]:
#history_re = np.load(model_re_path / 'HistoryArray.npy', allow_pickle = True)
#history_pre = np.load(model_pre_path / 'HistoryArray.npy', allow_pickle = True) 

In [7]:
def best_epoch(history):
  best_epch = 0
  best_loss = 10000
  for epoch in range(int(len(history)/32)):
    loss = 0
    tmp = history[epoch*32:(epoch+1)*32]
    #print(len(tmp))
    for element in tmp:
      loss += element.history['val_loss'][0]
    loss /= 32
    if loss < best_loss:
      best_loss = loss
      best_epoch = epoch
    
  return best_epoch

In [6]:
epoch_re = best_epoch(history_re)
epoch_pre = best_epoch(history_pre)

NameError: name 'history_re' is not defined

In [8]:
import tensorflow as tf
model_pre = tf.keras.models.load_model(model_pre_path / 'TrainedLSTM' / ('Epoch=' + str(1)))
model_re = tf.keras.models.load_model(model_re_path / 'TrainedLSTM' / ('Epoch=' + str(0)))


In [9]:
#target_path =  '/content/drive/MyDrive/Applications of Deep Learning - Medical Image Captioning/Models_V2/Pretrained Inception/'

In [10]:
# For pretrained model
word2Index = np.load(fetching_path / "word2Index.npy", allow_pickle=True).item()
index2Word = np.load(fetching_path / "index2Word.npy", allow_pickle=True).item()
variable_params = np.load(fetching_path / "variable_params.npy", allow_pickle=True).item()
test_captions = np.load(fetching_path / "test_captions.npy", allow_pickle=True).item()

test_features_pre = np.load(model_pre_path / "test_features_full.npy", allow_pickle=True).item()
test_features_re = np.load(model_re_path / "test_features_full.npy", allow_pickle=True).item()
print(len(test_features_pre), len(test_features_re))

In [11]:
keys_pre = list(test_features_pre.keys())
values_pre = np.array(list(test_features_pre.values()))

In [12]:
keys_re = list(test_features_re.keys())
values_re = np.array(list(test_features_re.values()))

In [13]:
def greedySearch(model, image_features, max_len, word2Index, index2Word):
    in_text = 'startseq'
    for i in range(max_len):
        sequence = [word2Index[w] for w in in_text.split() if w in word2Index]
        sequence = pad_sequences([sequence], maxlen=max_len, padding = 'post')
        
        y_pred = model.predict([image_features,sequence], verbose=0)
        y_pred = np.argmax(y_pred)
        
        word = index2Word[y_pred]
        in_text += ' ' + word
        
        if word == 'endseq':
            break

    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [20]:
def beam_search_predictions(model, image_features, max_len, word2Index, index2Word, beam_index):
    start = [word2Index["startseq"]]
    start_word = [[start, 1]]
    
    final_preds = []
    live_seqs = beam_index
    image_features = np.tile(image_features, (beam_index,1))
    errors = 0
    count = 0
    while len(start_word) > 0:
        #print(count)
        count+=1
        temp = []
        padded_seqs = []
        #Get padded seqs for each of the starting seqs so far, misnamed as start_word
        for s in start_word:
            par_caps = pad_sequences([s[0]], maxlen=max_len, padding='post')
            padded_seqs.append(par_caps)
        
        #Formatting input so that it can be used for a prediction
        padded_seqs = np.array(padded_seqs).reshape(len(start_word), max_len)
        
        preds = model.predict([image_features[:len(start_word)],padded_seqs], verbose=0)
        
        #Getting the best branches for each of the start seqs that we had
        for index, pred in enumerate(preds):
            word_preds = np.argsort(pred)[-live_seqs:]
            for w in word_preds:
                next_cap, prob = start_word[index][0][:], start_word[index][1]
                next_cap.append(w)
                prob *= pred[w]
                temp.append([next_cap, prob])
                
        start_word = temp
        # Sorting according to the probabilities
        start_word = sorted(start_word, reverse=False, key=lambda l: l[1])
        # Getting the top words from all branches
        start_word = start_word[-live_seqs:]
        
        for pair in start_word:
            #print(pair)
            #pair[0] = list(map(lambda x: x.replace(0, 12), pair[0]))
            errors += pair[0].count(0)
            pair[0] = [12 if x==0 else x for x in pair[0]]
            if index2Word[pair[0][-1]] == 'endseq':
                final_preds.append([pair[0][:-1], pair[1]])
                start_word = start_word[:-1]
                live_seqs -= 1
            if len(pair[0]) == max_len:
                final_preds.append(pair)
                start_word = start_word[:-1]
                live_seqs -= 1
    
    # Between all the finished sequences (either max len or predicted endseq), decide which is best
    max_prob = 0
    for index, pred in enumerate(final_preds):
        if pred[1] > max_prob:
            best_index = index
            max_prob = pred[1]
    
    # Convert to readable text
    final_pred = final_preds[best_index]
    final_caption = [index2Word[i] for i in final_pred[0]]
    final_caption = ' '.join(final_caption[1:])
    return final_caption, errors

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_prediction_single_image(model, image_features, max_len, word2Index, index2Word, method, beam_index = None): 
    if method == 'greedy':
        return greedySearch(model, image_features, max_len, word2Index, index2Word)
    else:
        if beam_index == None:
            print("For beam search you must input a beam_index. this is any natural number")
        else:
            return beam_search_predictions(model, image_features, max_len, word2Index, index2Word, beam_index)

In [24]:
import time

def generate_captions(model, values, model_path, variable_params, word2Index, index2Word, method, beam_index):
    true_captions = {}
    predicted_captions = {}

    for index, features in enumerate(values_re):

        predicted_caption = generate_prediction_single_image(model_re, features, 
                                     variable_params['max_caption_len'], word2Index, 
                                     index2Word, 'beam', beam_index = 5)


        true_caption = test_captions[keys_re[index]].split()[1:-1]
        true_caption = ' '.join(true_caption)

        true_captions[keys_re[index]] = true_caption
        predicted_captions[keys_re[index]] = predicted_caption
        print(index, predicted_caption)
        if index % 10 == 0:
          tmp = predicted_captions
          np.save(model_path / 'Predicted Captions', predicted_captions, allow_pickle = True)

    np.save(model_path / 'Predicted Captions', predicted_captions, allow_pickle = True)

In [25]:
# Generate captions for retrained model
generate_captions(model_re, values_re, model_re_path, variable_params, word2Index, index2Word, 'beam', beam_index = 2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


KeyboardInterrupt: 

In [ ]:
#Generate captions for pretrained model
generate_captions(model_pre, values_pre, model_pre_path, variable_params, word2Index, index2Word, 'beam', beam_index = 2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
